# Outlining the model


We have 10 inputs and 2 outputs, as there are only 2 possibilities.

We will build 2 hidden layers with 50 hidden units. This is easy to change later, but is a good starting point. 

## Import relevant libraries


In [2]:
import numpy as np
import tensorflow as tf

## Load the data

In [3]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs,test_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)


## Model

Copied code from MNIST lecture... it is the same with parameter changes and removal of the flattening element as we already did preprocessing.

In [9]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'), # activation function chosen because it is known to be good at this problem
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax') # output layer for this problem must turn outputs into probabilities, which softmax does.    
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size = 100
max_epochs = 100


early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs,
                           validation_targets),
          verbose=2)

Epoch 1/100
36/36 - 0s - loss: 0.5477 - accuracy: 0.7678 - val_loss: 0.4202 - val_accuracy: 0.8770
Epoch 2/100
36/36 - 0s - loss: 0.3558 - accuracy: 0.8768 - val_loss: 0.3120 - val_accuracy: 0.8837
Epoch 3/100
36/36 - 0s - loss: 0.3095 - accuracy: 0.8824 - val_loss: 0.2842 - val_accuracy: 0.8993
Epoch 4/100
36/36 - 0s - loss: 0.2908 - accuracy: 0.8894 - val_loss: 0.2806 - val_accuracy: 0.8971
Epoch 5/100
36/36 - 0s - loss: 0.2813 - accuracy: 0.8927 - val_loss: 0.2623 - val_accuracy: 0.9060
Epoch 6/100
36/36 - 0s - loss: 0.2700 - accuracy: 0.8944 - val_loss: 0.2541 - val_accuracy: 0.9083
Epoch 7/100
36/36 - 0s - loss: 0.2633 - accuracy: 0.8997 - val_loss: 0.2506 - val_accuracy: 0.9150
Epoch 8/100
36/36 - 0s - loss: 0.2572 - accuracy: 0.9003 - val_loss: 0.2433 - val_accuracy: 0.9172
Epoch 9/100
36/36 - 0s - loss: 0.2530 - accuracy: 0.9053 - val_loss: 0.2374 - val_accuracy: 0.9195
Epoch 10/100
36/36 - 0s - loss: 0.2497 - accuracy: 0.9039 - val_loss: 0.2521 - val_accuracy: 0.9083
Epoch 11/

In the original run, with 100 batch size and 100 epochs, the model looks like it has overfitted because the loss function is constantly reducing, but the val_loss is sometimes going up and sometimes going down. 

We probably need to set an early stopping procedure to fix this.

The early stopping variable is now in the code. By default, the Early Stopping object will monitor the validation loss and stop the training process the first time the validation loss starts increasing.

With the early stopping, there are only 7 epochs out of 100 before it stops. Sometimes you can allow 1 or 2 increases in validation losses to occur, in which case you can change the input to the early stopping object. The variable it takes is called patience, which lets us decide how many consecutive increases it tolerates.


With this model, we started out with a list of customers with 50% chance of guessing if they will buy another audiobook. With the model, we can predict with 90% accuracy whether a customer will buy or not buy another book.

## Test the model

In [10]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.2637 - accuracy: 0.9062


In [12]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))


Test loss: 0.26. Test accuracy: 90.62%


The final accuracy of the model is close to the validation accuracy. Great